In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
def get_soup(url, params = None, headers = None):
    response = requests.get(url, params = params,
                            headers = headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

def get_num_pages(url):
    soup = get_soup(url)
    pages = soup.find_all("div", class_ = "in-pagination__item")
    pages = [page.text for page in pages]
    return int(pages[-1])

In [3]:
def get_info_from_annuncio(annuncio):
    title = annuncio.find("a", class_ = "in-reListCard__title").text
    price = annuncio.find("div", class_ = "in-reListCardPrice").text
    infos = annuncio.find_all("div", class_ = "in-reListCardFeatureList__item")
    infos_text = [info.text for info in infos]
    res = {"title" : title, "price" : price, "infos":infos_text}
    return res

def get_immobiliare_info(url, params = None, headers = None):
    soup = get_soup(url, params= params, headers= headers)
    terreni = soup.find_all("div", class_ = "nd-mediaObject__content")
    res = []
    for i,terreno in enumerate(terreni):
        res.append(get_info_from_annuncio(terreno))
    return res

In [7]:
url = "https://www.immobiliare.it/vendita-terreni/lombardia/?criterio=rilevanza&superficieMinima=20000&idTipologia=107"
url_with_page_num = "https://www.immobiliare.it/vendita-terreni/lombardia/?criterio=rilevanza&superficieMinima=20000&idTipologia=107&pag={}"
num_pages = get_num_pages(url)
items = list()
for i in range(1,num_pages+1):
    items.extend(get_immobiliare_info(url_with_page_num.format(i)))

In [8]:
print(len(items))
items[-10:]

334


[{'title': 'Terreno edificabile via Sanguinetto, Centro, Quistello',
  'price': '€ 1.000',
  'infos': ['66.200 m²']},
 {'title': 'Terreno edificabile Strada Bugno 27, Viadana',
  'price': '€ 150.000',
  'infos': ['28.000 m²']},
 {'title': 'Terreno edificabile via Antonio Gramsci 1, Centro, Busto Garolfo',
  'price': '€ 1.350.000',
  'infos': ['22.500 m²']},
 {'title': "Terreno edificabile via Monte Grappa 11, Centro, Castel d'Ario",
  'price': '€ 950.000',
  'infos': ['26.200 m²']},
 {'title': 'Terreno edificabile Contrada Molino, Zecchini, Castel Goffredo',
  'price': '€ 2.578.050',
  'infos': ['30.330 m²']},
 {'title': 'Terreno edificabile via Ferdinando Magellano 8, Casone, Marcallo con Casone',
  'price': '€ 1.500.000',
  'infos': ['35.650 m²']},
 {'title': 'Terreno edificabile via Achille Grandi, Busto Garolfo',
  'price': 'Prezzo su richiesta',
  'infos': ['22.470 m²']},
 {'title': 'Terreno edificabile Strada Provinciale Arosio Canzo, Castelmarte',
  'price': '€ 1.111',
  'infos'